# Evaluate new_reward in sample_complete_laps

This notebook continues the analysis of [NewRewardAnalysis](NewRewardAnalysis.ipynb) and evaluating a new reward function
accros some sample laps obtained in [BulkAnalysisLapSelection](BulkAnalysisLapSelection.ipynb)

In [ ]:
import sys
sys.path.append('../')
import numpy
import pandas
import seaborn

from deepracer.tracks import TrackIO, Track

from reward import AWSReward
from utils.loader import get_df, load_sample_complete_laps
from utils.transformer import df_to_params, new_names_df, get_episode
from utils.new_rewards import aggregate_episode_reward, summary
track = TrackIO(base_path='../tracks').load_track("reinvent_base")

sample_complete_laps_path = '../logs/sample_complete_laps.csv'
scl_df = load_sample_complete_laps(sample_complete_laps_path)

Loaded 71 waypoints


## Define or load a new reward 

The function/object it should take a dictionary of params and return an object that can be casted to float

If you have a file with a function you can try to use the ``%%load`` magic to load it insted of copying it, just write ``%%load path/myfile.py`` in the first line of a cell an run it.

In [ ]:
class NewReward(AWSReward):
    max_speed    = 1.0
    
    def _calculateReward(self):
        return self.x + self.y

In [ ]:
cum_scl = aggregate_episode_reward(scl_df,NewReward, df_to_params, track, 'new_reward')

In [ ]:
cols = ['step','duration','lap_name','reward','new_reward']
seaborn.pairplot(cum_scl[cols],hue='lap_name',palette="husl")

In [ ]:
cols = ['stream','time','step','stream','speed','duration','reward','new_reward', 'lap_name']
aditional_map = {'lap_name': max}
summ_scl  = summary(cum_scl,['reward','new_reward'],use_stream=True,aditional_maps=aditional_map).reset_index()[cols]

In [ ]:
filtered_summ = pandas.concat( [ summ_scl.nlargest(5,'duration'),
                                summ_scl.nsmallest(5,'duration')
                               ])
filtered_summ

In [ ]:
seaborn.pairplot(filtered_summ,hue='lap_name')

In [ ]:
seaborn.pairplot(summ_scl,hue='lap_name')

In [ ]:
laps = ['training-simulation-logs-z6.log_30','training-simulation-logs-w5.log_39']
filtered_by_lap_summ = summ_scl[summ_scl.lap_name.isin(laps)]
filtered_by_lap_summ

In [ ]:
seaborn.pairplot(filtered_by_lap_summ,hue='lap_name')